In [37]:
import pandas as pd
import numpy as np

In [38]:
first_data = pd.read_csv('data/1995_Street_Tree_Census.csv')
second_data = pd.read_csv('data/2005_Street_Tree_Census.csv')
third_data = pd.read_csv('data/2015_Street_Tree_Census.csv')

/Users/kateharwood/opt/miniconda3/envs/climate/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Standardize feature columns across datasets

In [39]:
# Extract the columns we care about
first_data = first_data[["Latitude", "Longitude", "Borough", "Postcode_Original", "Spc_Latin", "Diameter", "Condition"]]
second_data = second_data[["latitude", "longitude", "boroname", "zipcode", "spc_latin", "tree_dbh", "status"]]
third_data = third_data[["latitude", "longitude", "borough", "postcode", "spc_latin", "tree_dbh", "status", "health"]]

In [40]:
# Remove NaN rows 
# TODO should I remove all NaN rows from all datasets?
# TODO is this actually doing anything now? that it is changed to postcode original
first_data = first_data[first_data['Postcode_Original'].notna()]


In [41]:
first_data = first_data.rename(columns={"Postcode_Original": "zipcode", "Borough": "borough", "Spc_Latin": "spc_latin", "Diameter": "tree_diameter"})
second_data = second_data.rename(columns={"boroname": "borough", "tree_dbh": "tree_diameter"})
second_data.borough[second_data['borough'] == 5] = 'Staten Island'
third_data = third_data.rename(columns={'postcode': 'zipcode'})


/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_14706/3548259356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.borough[second_data['borough'] == 5] = 'Staten Island'


In [42]:
first_data = first_data[first_data['spc_latin'].notna()]
first_data['spc_latin'] = first_data['spc_latin'].str.lower()
second_data = second_data[second_data['spc_latin'].notna()]
second_data['spc_latin'] = second_data['spc_latin'].str.lower()


# Standardize all health statuses across the 2005 and 2015 datasets

In [43]:
# Rename "Excellent" to "Good", rename "Good" to "Fair"
second_data.status[second_data['status'] == "Good"] = "Fair"
second_data.status[second_data['status'] == "Excellent"] = "Good"
second_data = second_data.rename(columns={"status": "health"})
second_data.health.unique()



/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_14706/16073105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.status[second_data['status'] == "Good"] = "Fair"
/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_14706/16073105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_data.status[second_data['status'] == "Excellent"] = "Good"


array(['Fair', 'Poor', 'Good', 'Dead'], dtype=object)

In [44]:
# Set all status to be "Alive" or "Dead" (remove "Stump")
third_data.health[(third_data['health'].isna()) & (third_data['status'] != "Alive")] = "Dead"
# Remove the row with status == alive and health is na
third_data.drop(third_data[third_data['health'].isna()].index, inplace = True)
# Drop extra status column
third_data.drop(columns=['status'], inplace=True)


/var/folders/z7/x5j_gv2x6f92wlfl832cwwy80000gn/T/ipykernel_14706/3319506031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  third_data.health[(third_data['health'].isna()) & (third_data['status'] != "Alive")] = "Dead"


In [45]:
# Make categorical health data numeric
health_dict = {'Dead':0, 'Poor':1, 'Fair':2, 'Good':3}
second_data['health_status'] = second_data['health'].map(health_dict)
third_data['health_status'] = third_data["health"].map(health_dict)
second_data

,latitude,longitude,borough,zipcode,spc_latin,tree_diameter,health,health_status
0,40.632653,-74.000245,Brooklyn,11219,pyrus calleryana,6,Fair,2
1,40.620084,-73.901453,Brooklyn,11234,platanus acerifolia,6,Fair,2
2,40.617996,-73.899111,Brooklyn,11234,acer platanoides crimson king,13,Fair,2
3,40.619694,-73.901003,Brooklyn,11234,platanus acerifolia,13,Fair,2
4,40.618323,-73.899467,Brooklyn,11234,platanus acerifolia,15,Fair,2
...,...,...,...,...,...,...,...,...
592367,40.586260,-74.148797,Staten Island,10314,platanus acerifolia,9,Fair,2
592368,40.586090,-74.149013,Staten Island,10314,acer saccharinum,7,Fair,2
592369,40.585802,-74.149156,Staten Island,10314,acer platanoides crimson king,5,Fair,2
592370,40.585802,-74.149156,Staten Island,10314,acer platanoides crimson king,9,Fair,2


# Group trees into geospatial segments that match across datasets

In [46]:
# Group trees into geospatial segments by lat and long
# step = 0.00000001 # This should be about 1/4-1/2 a city block 
# step = 0.001
step = 0.0005 # Actually no this should be about 1/2 block ish, TODO
to_bin = lambda x: np.floor(x / step) * step
second_data["latBin"] = to_bin(second_data.latitude)
second_data["lonBin"] = to_bin(second_data.longitude)

# Drop all rows where lat or long is 0
second_data.drop(second_data[second_data['latBin'] == 0].index, inplace = True)
second_data.drop(second_data[second_data['lonBin'] == 0].index, inplace = True)


In [47]:
# Only keep the rows whose zipcodes overlap across datasets
zipcodes_intersection = set(first_data['zipcode']).intersection(set(second_data['zipcode'])).intersection(third_data['zipcode'])
first_data = first_data[first_data['zipcode'].isin(zipcodes_intersection)]
second_data = second_data[second_data['zipcode'].isin(zipcodes_intersection)]
third_data = third_data[third_data['zipcode'].isin(zipcodes_intersection)]


In [48]:
# Set latBin and lonBin values to match across second and third datasets
from scipy.spatial.distance import cdist

third_data["latBin"] = 0
third_data["lonBin"] = 0
second_data = second_data.sort_values('zipcode')
third_data = third_data.sort_values('zipcode')

latBins = []
lonBins = []
for zipcode in sorted(list(zipcodes_intersection)):
    test2 = second_data[second_data['zipcode'] == zipcode]
    test3 = third_data[third_data['zipcode'] == zipcode]
    temp = test2[['latBin','lonBin']].iloc[np.argmin(cdist(test3[['latitude','longitude']], test2[['latBin','lonBin']], metric='euclidean'), axis=1)]
    latBins.extend(list(temp.latBin))
    lonBins.extend(list(temp.lonBin))
third_data['latBin'] = latBins
third_data['lonBin'] = lonBins


In [49]:
# Do the same for the first data
first_data["latBin"] = 0
first_data["lonBin"] = 0
second_data = second_data.sort_values('zipcode')
first_data = first_data.sort_values('zipcode')

latBins = []
lonBins = []
for zipcode in sorted(list(zipcodes_intersection)):
    test2 = second_data[second_data['zipcode'] == zipcode]
    test1 = first_data[first_data['zipcode'] == zipcode]
    temp = test2[['latBin','lonBin']].iloc[np.argmin(cdist(test1[['Latitude','Longitude']], test2[['latBin','lonBin']], metric='euclidean'), axis=1)]
    latBins.extend(list(temp.latBin))
    lonBins.extend(list(temp.lonBin))
first_data['latBin'] = latBins
first_data['lonBin'] = lonBins


In [50]:
# Drop the rows whose actual lat and long differ too much from their latBin and lonBin

third_data['lonDistance'] = (third_data['lonBin']-third_data['longitude']).abs()
third_data['latDistance'] = (third_data['latBin']-third_data['latitude']).abs()

third_data.drop(third_data[third_data['health'].isna()].index, inplace = True)

third_data.drop(third_data[third_data['lonDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia
third_data.drop(third_data[third_data['latDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia


first_data['lonDistance'] = (first_data['lonBin']-first_data['Longitude']).abs()
first_data['latDistance'] = (first_data['latBin']-first_data['Latitude']).abs()

first_data.drop(first_data[first_data['lonDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia
first_data.drop(first_data[first_data['latDistance'] > 0.001].index, inplace = True) #"neighborhood or street" from wikipedia


# Merge the three datasets 
### Join the 1995 (first_data) features to the 2005 (second_data) health status values (matching on latBin and lonBin)
### Join the 2005 features to the 2015 health status values (matching on latBin and lonBin)

In [51]:
# Average the health data based on latBin/lonBin groups (and round to nearest whole number aka nearest health category)
second_data['avg_health_round'] = second_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean').round(0)
third_data['avg_health_round'] = third_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean').round(0)
second_data['avg_health'] = second_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean')
third_data['avg_health'] = third_data.groupby(["latBin", "lonBin"])['health_status'].transform('mean')

In [53]:
# Keep only necessary features and merge dataframes

second_data_health = second_data.drop(columns={'tree_diameter', 'borough', 'health', 'health_status', 'latitude', 'longitude', 'spc_latin', 'zipcode'})
first_data_features = first_data.drop(columns={'Latitude', 'Longitude', 'Condition'})
second_data_health = second_data_health.drop_duplicates(subset=['avg_health', 'avg_health_round', 'latBin','lonBin'])

merged_first_second = first_data_features.merge(second_data_health, on=['latBin', 'lonBin'], how="inner")
merged_first_second

third_data_health = third_data.drop(['borough', 'zipcode', 'spc_latin', 'tree_dbh', 'health', 'health_status', 'latitude', 'longitude'], axis=1)
second_data_features = second_data.drop(columns=['avg_health', 'avg_health_round', 'health', 'health_status', 'latitude', 'longitude'])
third_data_health = third_data_health.drop_duplicates(subset=['avg_health','avg_health_round', 'latBin','lonBin'])

merged_second_third = second_data_features.merge(third_data_health, on=['latBin', 'lonBin'], how="inner")
merged_second_third

# TODO note that the species still don't match up

,borough,zipcode,spc_latin,tree_diameter,latBin,lonBin,lonDistance,latDistance,avg_health_round,avg_health
0,Manhattan,10001,tilia cordata,6,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
1,Manhattan,10001,robinia pseudoacacia,8,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
2,Manhattan,10001,quercus palustris,9,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
3,Manhattan,10001,platanus acerifolia,14,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
4,Manhattan,10001,zelkova serrata,5,40.749,-73.9995,1.634000e-05,0.000196,2.0,2.111111
...,...,...,...,...,...,...,...,...,...,...
544364,Queens,11694,platanus acerifolia,16,40.581,-73.8530,1.771800e-04,0.000005,3.0,3.000000
544365,Queens,11694,acer platanoides,11,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545
544366,Queens,11694,acer platanoides,14,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545
544367,Queens,11694,acer platanoides,14,40.579,-73.8455,8.304000e-05,0.000279,2.0,2.454545


In [55]:
# Save dataset for use in models
final_data = merged_first_second.append(merged_second_third)
final_data.to_csv('data/parsed_data.csv')


In [470]:
# second_data_features.sort_values(['latBin', 'lonBin']).head(20)
# merged_second_third.sort_values('avg_health')
# merged_second_third.avg_health.unique()
# third_data_health.avg_health.unique()
# # second_data_health.avg_health.unique()
# third_data_health[third_data_health['latBin'] == 40.9110].sort_values(['latBin', 'lonBin'])
# third_data_health.sort_values(['latBin', 'lonBin']).tail(83)

# merged_second_third[merged_second_third['borough'].isna()]


In [484]:
# second_data_features.sort_values(['latBin', 'lonBin']).tail(50)

In [485]:
# third_data_health.sort_values(['latBin', 'lonBin']).head(20)

In [ ]:
# df['avg_result'] = df.groupby(['a', 'b'])['result'].transform('mean')
# groups.size().sort_values().value_counts()
# groups.size().sort_values()

# third_data.sort_values(by='latBin', ascending=False).head(50)

# second_data.sort_values(by='latBin', ascending=False).head(10)

# third_data.sort_values(by='latBin', ascending=False)
# third_data[third_data['latBin'] > 40.912].sort_values(by='latBin', ascending=False)

In [ ]:
# third_data = pd.read_csv('../data/2015_data.csv')
# third_data.drop(columns=['Unnamed: 0'], inplace = True)
# third_data

# first_data = pd.read_csv('../data/1995_data.csv')
# first_data.drop(columns=['Unnamed: 0'], inplace = True)

In [ ]:
# Do the same for the 1995 data
# Take 13ish mins
# TODO nope

# first_data["latBin"] = 0
# first_data["lonBin"] = 0
# binValues = sorted(list(dict.fromkeys(list(zip(second_data['latBin'], second_data['lonBin'])))))

# for latBinValue, lonBinValue in binValues:
#     first_data.loc[first_data['Latitude'] >= latBinValue, ['latBin', 'lonBin']] = latBinValue, lonBinValue

# for latBinValue in sorted(second_data['latBin'].unique()):
#     first_data.loc[first_data['Latitude'] >= latBinValue, ['latBin']] = latBinValue

# for lonBinValue in sorted(second_data['lonBin'].unique()):
#     first_data.loc[first_data['Longitude'] >= lonBinValue, ['lonBin']] = lonBinValue



In [ ]:
# third_data[third_data['zipcode'] == zipcode]['latBin'] = latBins
# third_data[third_data['zipcode'] == zipcode]['lonBin'] = lonBins
# third_data
# second_data[['latBin','lonBin']] = \
#   second_data[['latBin','lonBin']].iloc[np.argmin(cdist(third_data[['latitude','longitude']], second_data[['latBin','lonBin']],
#   metric='euclidean' ), axis=1),:].copy().reset_index(drop=True)
# third_data
# test3 = third_data.head(5000)
# test2 = second_data.head(5000)
# np.argmin(cdist(test3[['latitude','longitude']], test2[['latBin','lonBin']], metric='euclidean'))
# cdist(test3[['latitude','longitude']], test2[['latBin','lonBin']], metric='euclidean')

# test2 = second_data.head(5)
# test3 = third_data.head(5)
# # print(test2)
# # print(test3)
# test3[['latBin', 'lonBin']] = test2[['latBin','lonBin']].iloc[np.argmin(cdist(test3[['latitude','longitude']], test2[['latBin','lonBin']], metric='euclidean'), axis=1)].copy().reset_index(drop=True)
# test3
# third_data.sort_values('zipcode')


In [ ]:
# second_data.head(100)
# groups = second_data.groupby(['latBin', 'lonBin'])

# # groups.size().sort_values().tail(50)

# groups.size()

In [ ]:
# Set latBin and lonBin values to match across datasets
# Run this once and save output, it takes a while (22ish mins)
# TODO this is not true

# from collections import defaultdict

# third_data["latBin"] = 0
# third_data["lonBin"] = 0
# binValues = sorted(list(dict.fromkeys(list(zip(second_data['latBin'], second_data['lonBin'])))))
# len(binValues)
# len(second_data['latBin'].unique())
# len(second_data['lonBin'].unique())


# for latBinValue, lonBinValue in binValues:
#     third_data.loc[third_data['latitude'] >= latBinValue, ['latBin']] = latBinValue
#     third_data.loc[third_data['longitude'] >= lonBinValue, ['lonBin']] = lonBinValue

# binValues = dict.fromkeys(list(second_data['latBin'].unique()))
# binValues = defaultdict(list)

# for k, v in list(zip(second_data['latBin'], second_data['lonBin'])):
#     if v not in binValues[k]:
#         binValues[k].append(v)

# binValues.keys()
# for latBinValue in sorted(binValues.keys()):
#     third_data.loc[third_data['latitude'] >= latBinValue, ['latBin']] = latBinValue
#     for lonBinValue in sorted(binValues[latBinValue]):
#         third_data.loc[(third_data['latBin'] == latBinValue) & (third_data['longitude'] <= lonBinValue), ['lonBin']] = lonBinValue # "less than" bc the values are negative

# for lonBinValue in sorted(second_data['lonBin'].unique()):
#     third_data.loc[third_data['longitude'] >= lonBinValue, ['lonBin']] = lonBinValue

#TODO do the min euclidean distance
# third_data.loc[third_data['latitude'] >= latBinValue

# third_data

In [ ]:
# Save the processed dataframes so we don't have to compute lat and long bins every time
# TODO not true anymore
third_data.to_csv('../data/2015_data.csv')
# first_data.to_csv('../data/1995_data.csv')